In [1]:
import os, time, torch, numpy as np
from torch.utils.tensorboard.writer import SummaryWriter
from src.TorchDSP.dataloader import MyDataset 
from src.TorchDSP.eq import eqAMPBC, eqPBC, eqPBC_step, MultiStepPBC, eqAMPBC_step
from src.TorchDSP.loss import MSE, Qsq
from src.TorchSimulation.receiver import BER



window_size = 101
TBPL = 100000
Nwindow = 1
train_data = MyDataset('dataset_A800/train.h5', Nch=[21], Rs=[80], Pch=[2], 
                       window_size=window_size + TBPL - 1, strides=TBPL, Nwindow=Nwindow, truncate=20000, 
                       Tx_window=True, pre_transform='Rx_CDCDDLMS(taps=32,lr=[0.015625, 0.0078125])')

test_data = MyDataset('dataset_A800/test.h5', Nch=[21], Rs=[80], Pch=[2], 
                       window_size=window_size + TBPL - 1, strides=TBPL, Nwindow=Nwindow, truncate=20000, 
                       Tx_window=True, pre_transform='Rx_CDCDDLMS(taps=32,lr=[0.015625, 0.0078125])')

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, drop_last=True)

print('Train Data number:',len(train_data))
print('Test Data number:',len(test_data))

def Test(net, dataloader):
    net.eval()
    mse, power, ber, N = 0,0,0, len(dataloader)
    with torch.no_grad():
        for Rx, Tx, info in dataloader:
            Rx, Tx, info = Rx.cuda(), Tx.cuda(), info.cuda()
            symb = Tx[:,net.overlaps//2:-net.overlaps//2]
            PBC = net(Rx, info)
            mse += MSE(PBC, symb).item()
            power += MSE(symb, 0).item() 
            ber += np.mean(BER(PBC, symb)['BER'])
    return {'MSE':mse/N, 'SNR': 10*np.log10(power/mse), 'BER':ber/N, 'Qsq': Qsq(ber/N)}

def write_log(writer, epoch, train_loss, metric):
    writer.add_scalar('Loss/train',  train_loss, epoch)
    writer.add_scalar('Loss/Test', metric['MSE'], epoch)
    writer.add_scalar('Metric/SNR', metric['SNR'], epoch)
    writer.add_scalar('Metric/BER', metric['BER'], epoch)
    writer.add_scalar('Metric/Qsq', metric['Qsq'], epoch)
    print(epoch, 'Train MSE:',  train_loss, 'Test MSE:', metric['MSE'],  'Qsq:', metric['Qsq'], flush=True)
    

Train Data number: 1
Test Data number: 1


## LBFGS

In [2]:
log_path = '_outputs/log_test/PBC_step_LBFGS'
os.makedirs(log_path, exist_ok=True)
writer = SummaryWriter(log_path)
epochs = 10

train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, drop_last=True)
data_iter = iter(train_loader)
Rx,Tx,info = next(data_iter)
Rx, Tx, info = Rx.cuda(), Tx.cuda(), info.cuda()

# net = MultiStepPBC(2, window_size, index_type='reduce-2', pol_share=True)
net = eqPBC_step(window_size, index_type='reduce-2', pol_share=True)
# net = eqAMPBC_step(window_size, 1, fwm_share=True)
net.cuda()
optimizer = torch.optim.LBFGS(net.parameters(), lr=1)

def closure():
    optimizer.zero_grad()
    PBC = net(Rx, info)
    loss = MSE(PBC, Tx[:,net.overlaps//2:-net.overlaps//2])
    loss.backward()
    return loss


for epoch in range(epochs):
    t0 = time.time()
    loss = optimizer.step(closure)
    t1 = time.time() 
    print('Time per epoch: ', t1 - t0, flush=True)
    write_log(writer, epoch, loss.item(), Test(net, test_loader))

Time per epoch:  2.0797057151794434
0 Train MSE: 0.03528790920972824 Test MSE: 0.03243440389633179 Qsq: 7.9560107776447255
Time per epoch:  0.1890885829925537
1 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.0969095230102539
2 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.09644460678100586
3 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.09630608558654785
4 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.09744429588317871
5 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.0966196060180664
6 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.09613180160522461
7 Train MSE: 0.032690130174160004 Test MSE: 0.0324343740940094 Qsq: 7.955516123994853
Time per epoch:  0.

## Adam

In [ ]:
os.makedirs('_outputs/log_test/PBC_step_adam', exist_ok=True)
writer = SummaryWriter('_outputs/log_test/PBC_step_adam')
epochs = 30

train_loader = torch.utils.data.DataLoader(train_data, batch_size=10000, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=True, drop_last=True)

net = eqAMPBC(M=window_size, rho=1)
net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(epochs):
    train_loss = 0
    for i, (Rx, Tx, info) in enumerate(train_loader):
        Rx, Tx, info = Rx.cuda(), Tx.cuda(), info.cuda()
        y = net(Rx)
        loss = MSE(y, Tx)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  
        writer.add_scalar('Loss/train_batch', loss.item(), epoch*len(train_loader)+i)
    scheduler.step()
    metric = Test(net, test_loader)
    write_log(writer, epoch, train_loss/len(train_loader), metric)
